In [37]:
import pandas as pd
import numpy as np


In [ ]:
df = pd.read_csv("Tesla_Trustpilot_Reviews.csv")
df.insert(0, 'id', range(1, len(df) + 1))
df.head(10)

,id,Username,Title,Content,Rating,Date
0,1,John McLean,Utter Rubbish.,Utter Rubbish.3 owner for the last 3 years. Go...,1,2024-11-19T06:55:16.000Z
1,2,Angry from Manchester,5/5 for the car. 0/5 for the dealer,"The car (model 3) is excellent, though feels a...",3,2024-11-18T14:58:54.000Z
2,3,Kevin O,Musk is the most embarrassing man that…,Musk is the most embarrassing man that has eve...,1,2024-11-21T09:13:18.000Z
3,4,Andrew Haworth,Put a deposit on a cybertruck 4 years…,Put a deposit on a cybertruck 4 years ago. For...,1,2024-11-20T18:43:26.000Z
4,5,Ahmed Radm,Thanks for the office tomorrow so I can…,Thanks for the office tomorrow so I can do it ...,5,2024-11-23T01:27:42.000Z
5,6,Daniel,Zero Customer Service,Cancelled my Model 3 order due to unforeseen c...,1,2024-11-19T15:25:18.000Z
6,7,Gill Beddows,John at Solihull,John at Solihull - fantastic help which was no...,5,2024-11-15T17:40:28.000Z
7,8,Ben,Absolutely shocking customer service,Absolutely shocking customer service. Contact ...,1,2024-11-14T19:23:27.000Z
8,9,Customer,Tesla after must be one of the worst…,Tesla after must be one of the worst company I...,1,2024-11-11T08:15:58.000Z
9,10,James Cudd,Tesla UK Attempting to Return Car After Diagno...,My leased Tesla experienced a complete steerin...,1,2024-11-22T20:14:11.000Z


In [39]:
#On verifie les ligne vide
df.isnull().sum()

id           0
Username     0
Title        0
Content     18
Rating       0
Date         0
dtype: int64

In [40]:
#On suprimme lesnligne vide de la colonne content
df = df.dropna(subset=['Content'])
df.isnull().sum()

id          0
Username    0
Title       0
Content     0
Rating      0
Date        0
dtype: int64

In [41]:
#On verifie le type de chaque colonne
df.dtypes

id           int64
Username    object
Title       object
Content     object
Rating       int64
Date        object
dtype: object

# Model de classification Hugging Face

In [42]:
'''
label2emotion = {
    '1 star': "très négatif",
    '2 stars': "négatif",
    '3 stars': "neutre",
    '4 stars': "positif",
    '5 stars': "très positif"
}
'''

'\nlabel2emotion = {\n    \'1 star\': "très négatif",\n    \'2 stars\': "négatif",\n    \'3 stars\': "neutre",\n    \'4 stars\': "positif",\n    \'5 stars\': "très positif"\n}\n'

In [ ]:
import pandas as pd
from transformers import pipeline

# Load the dataset
file_path = 'Tesla_Trustpilot_Reviews.csv'
df = pd.read_csv(file_path)

# Initialize the sentiment analysis pipeline for multi-class labels
analyzer = pipeline(
    task='text-classification',
    model="nlptown/bert-base-multilingual-uncased-sentiment",
    tokenizer="nlptown/bert-base-multilingual-uncased-sentiment"
)

# Define the label-to-emotion mapping for 5 labels
label2emotion = {
    '1 star': "très négatif",
    '2 stars': "négatif",
    '3 stars': "neutre",
    '4 stars': "positif",
    '5 stars': "très positif"
}

# Function to perform sentiment analysis
def MultiClassSentimentAnalysis(text):
    result = analyzer(text, return_all_scores=True)
    max_result = max(result[0], key=lambda x: x['score'])
    label = label2emotion[max_result['label']]  # Map the label to emotion
    prob = round(max_result['score'] * 100, 2)  # Convert probability to percentage
    return label, prob

# Initialize lists to store sentiment analysis results
sentiment_label = []
sentiment_prob = []

# Process the 'Content' column and perform sentiment analysis
count = 0
for i, row in df.iterrows():
    count += 1
    text = str(row['Content'])
    if len(text) > 512:
        text = text[:510]  # Truncate to fit model input size
    if len(text) < 2:
        text += "N/A"  # Handle very short or empty reviews
    result = MultiClassSentimentAnalysis(text)
    sentiment_label.append(result[0])
    sentiment_prob.append(result[1])
    print(f"Processing review {count}/{len(df)}")

# Add sentiment analysis results to the DataFrame
df['sentiment_label'] = sentiment_label
df['sentiment_prob'] = sentiment_prob

# Save the updated DataFrame to a new file
output_path = 'Tesla_Trustpilot_Reviews_Analyzed.csv'
df.to_csv(output_path, index=False)
print(f"Sentiment analysis completed. Results saved to {output_path}.")


C:\Users\moham\AppData\Roaming\Python\Python311\site-packages\huggingface_hub\file_download.py:139: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\moham\.cache\huggingface\hub\models--nlptown--bert-base-multilingual-uncased-sentiment. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)
Some layers from the model checkpoint at nlptown/bert-base-multilingual-unca

Processing review 1/1000
Processing review 2/1000
Processing review 3/1000
Processing review 4/1000
Processing review 5/1000
Processing review 6/1000
Processing review 7/1000
Processing review 8/1000
Processing review 9/1000
Processing review 10/1000
Processing review 11/1000
Processing review 12/1000
Processing review 13/1000
Processing review 14/1000
Processing review 15/1000
Processing review 16/1000
Processing review 17/1000
Processing review 18/1000
Processing review 19/1000
Processing review 20/1000
Processing review 21/1000
Processing review 22/1000
Processing review 23/1000
Processing review 24/1000
Processing review 25/1000
Processing review 26/1000
Processing review 27/1000
Processing review 28/1000
Processing review 29/1000
Processing review 30/1000
Processing review 31/1000
Processing review 32/1000
Processing review 33/1000
Processing review 34/1000
Processing review 35/1000
Processing review 36/1000
Processing review 37/1000
Processing review 38/1000
Processing review 39/